# Volumetric Stiffness Field Playground

In [1]:
# Auto reload modules
%load_ext autoreload
%autoreload 2

#if you haven't installed vsf and just want to run this from the current directory, uncomment the following lines
import sys
sys.path.append('..')

import numpy as np
from vsf import vsf_from_file
from klampt.io import open3d_convert
from klampt import vis, Geometry3D, WorldModel
from vsf.visualize.klampt_visualization import vsf_show

vis.init('PyQt')  #needed to pop up Klampt OpenGL windows in Jupyter notebook
None

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
***  klampt.vis: using Qt6 as the visualization backend  ***


### Download the Demo Data

Download the demo data from the following link:  
**https://uofi.box.com/s/31wozq63qgqvg8012r1jdfj5mdchmmfp**

After extracting, place the `demo_data` folder in the `openvsf` directory as `openvsf/demo_data`.

## Loading a VSF from file

In this example we will load one of the demo objects.  Let's assume that you have placed `demo_data` in the parent directory of this Jupyter notebook.

In [2]:
vsf_model = vsf_from_file('../demo_data/saved_vsfs/rubber_fig_tall_angle00/vsf.npz')
print('Model loaded')

Model loaded


Now we can visualize the loaded model.  We can do so in Open3D using a point cloud visualization, or in Klampt with a point cloud or implicit surface visualization.

In [3]:
# Visualize VSF point cloud in Open3D
import open3d as o3d
from vsf.visualize.o3d_visualization import vsf_to_point_cloud
vis_pcd = vsf_to_point_cloud(vsf_model)
o3d.visualization.draw_geometries([vis_pcd], window_name='VSF vis_pcd')

Stiffness range:  -3.0433426e-18 40.33515


In [ ]:
# Klampt point cloud visualization of VSF
from vsf.visualize.klampt_visualization import vsf_to_point_cloud
k_pcd = Geometry3D(vsf_to_point_cloud(vsf_model))
vis.debug(k_pcd)

Stiffness range:  -3.0433426e-18 40.33515
################################################################
klampt.vis: Running multi-threaded dialog, waiting to complete...
vis: creating GL window
######### QtGLWindow setProgram ###############
######### QtGLWindow setProgram ###############
#########################################
klampt.vis: Dialog on window 1 starting
#########################################
######### QtGLWindow Initialize GL ###############
QtGLWidget.initialize: no action menu?


QtGLWindow.resizeGL: called when invisible?
QtGLWindow.paintGL: called while invisible?
#########################################
klampt.vis: Dialog done on window 1 result 0
#########################################
klampt.vis: ... dialog done, leaving thread open
################################################################


vis: creating GL window
######### QtGLWindow setProgram ###############
#########################################
klampt.vis: Making window 0
#########################################
######### QtGLWindow Initialize GL ###############
QtGLWindow.resizeGL: called when invisible?
QtGLWindow.paintGL: called while invisible?
#########################################
klampt.vis: Window 0 close
#########################################
vis: creating GL window
######### QtGLWindow setProgram ###############
######### QtGLWindow setProgram ###############
#########################################
klampt.vis: Dialog on window 2 starting
#########################################
######### QtGLWindow Initialize GL ###############
QtGLWidget.initialize: no action menu?
QtGLWindow.resizeGL: called when invisible?
QtGLWindow.paintGL: called while invisible?
#########################################
klampt.vis: Dialog done on window 2 result 0
#########################################
vis: creating G

In [5]:
# Klampt volumetric visualization of VSF
stiffness_levels = [0.27072093, 4.54502328, 8.81932563, 9.09362797, 10.36793032]
vsf_show(vsf_model, stiffness_levels)

Setting voxel grid resolution 0.021477368597769594
klampt.vis: auto-fitting camera to scene.


Unrecognized OpenGL version
Unrecognized OpenGL version


## VSF estimation from tactile data

In this example, we will create VSF from a dataset that includes tactile observations.

- Create an empty VSF.  We will use a VSFFactory to shape the VSF rest points from an RGBD point cloud
- Configure the robot model and sensor(s).  In this case the robot is a Kinova Gen3, and there is only one sensor that provides joint torques.
- Load a dataset of sequences giving robot actions and observations from sensor(s) 
- Create an estimator, and run it in batch mode on the dataset.

Let's start by creating an empty VSF.  If you have an RGB-D scanned object, you should use a `VSFRGBDCameraFactory` like the following code to construct the VSF points from a region of interest in the point cloud, extending through a volume behind the camera's point of view.  It doesn't really matter how you get the points; you can also create an empty box using the `vsf.vsf_from_box()` function, create a VSF from a 3D scanned mesh using `vsf.vsf_from_mesh()`, or load a previously saved VSF from disk.

In [6]:
from vsf.core.vsf_factory import VSFRGBDCameraFactory, VSFRGBDCameraFactoryConfig, ViewConfig

view = ViewConfig(origin=[-0.55639158,1.04689234,0.52593784])  # need to set the origin of the camera so that volume points can be created
#we will select points within a bounding box 
config = VSFRGBDCameraFactoryConfig(features=['color'], voxel_size=0.02,
                                    view = view, bbox = [[-1.03, -0.64, -0.22], [-0.29, 0.23, 0.73]], downsample_visual=True, verbose=True)
factory = VSFRGBDCameraFactory(config)
#process() does all the work
vsf_empty = factory.process('../demo_data/datasets/rubber_fig_tall_angle00/object/bg_pcd.pcd')  #the PCD file is the point cloud of the RGBD scene, including the background 
print('{} rest points for VSF model created from RGBD factory'.format(len(vsf_empty.rest_points)))

BBox selection: keeping 129694 points out of 459947
downsampling visual point cloud to voxel size 0.02
iter step 0 left pts: 129694
iter step 1 left pts: 129666
iter step 2 left pts: 129567
iter step 3 left pts: 129479
iter step 4 left pts: 129241
iter step 5 left pts: 129014
iter step 6 left pts: 128862
iter step 7 left pts: 128657
iter step 8 left pts: 128345
iter step 9 left pts: 128086
iter step 10 left pts: 127695
iter step 11 left pts: 127290
iter step 12 left pts: 126953
iter step 13 left pts: 126644
iter step 14 left pts: 126349
iter step 15 left pts: 126033
iter step 16 left pts: 125654
iter step 17 left pts: 125288
iter step 18 left pts: 124893
iter step 19 left pts: 124478
iter step 20 left pts: 124064
iter step 21 left pts: 123677
iter step 22 left pts: 123282
iter step 23 left pts: 122795
iter step 24 left pts: 121882
iter step 25 left pts: 120518
iter step 26 left pts: 118863
iter step 27 left pts: 117332
iter step 28 left pts: 114992
iter step 29 left pts: 110272
iter st

In [7]:
# Show the processed VSF model
from vsf.visualize.klampt_visualization import vsf_to_point_cloud
vis.debug(empty_pc = vsf_to_point_cloud(vsf_empty, masked_view_fraction=1.0))

Stiffness range:  0.0 0.0
#########################################
klampt.vis: Creating dialog on window 2
#########################################
vis.dialog(): waiting for window 2 on vis thread to complete....
vis.dialog(): ... dialog done


Next, we create a world with a robot, a simulator, and one or more sensors.  This example loads a Kinova Gen3 robot model and uses the joint torque sensors in the simulator.  We also add the empty VSF into the simulator using `sim.add_deformable`.  This creates a body that can be moved through space.  (By default, all VSFs are considered to have a static pose, which works well if the object doesn't move much during interaction.  We will consider moving objects later.)

In [8]:
from vsf.sim import klamptWorldWrapper, QuasistaticVSFSimulator
from vsf.sim.point_vsf_body import ContactParams
from vsf.sensor.joint_torque_sensor import JointTorqueSensor

#create a world
world = klamptWorldWrapper()
world.add_robot('kinova','../knowledge/robot_model/kinova_gen3.urdf')
robot = world.world.robot(0)

#this preprocessing needs to be done before running the simulator with a point VSF body
world.setup_local_pcd_lst('open3d')
world.setup_local_sdf_lst()

#create a simulator with the world, a joint torque sensor, and the vsf body
sensor = JointTorqueSensor('kinova_joint_torques','kinova',[robot.link(i).name for i in range(1,8)])
sim = QuasistaticVSFSimulator(world, [sensor])
vsf_body = sim.add_deformable('vsf',vsf_empty, contact_params=None)  #if you want to customize how the VSF is simulated, you can pass in a ContactParams object here

WorldModel::LoadRobot: ../knowledge/robot_model/kinova_gen3.urdf
URDFParser: Link size: 9
URDFParser: Joint size: 9
URDFParser: Done loading robot file ../knowledge/robot_model/kinova_gen3.urdf
FMM_Fill identifies 1305 surface, 731 interior, 2003 exterior cells
FMM starting with 835 surface cells, grid of size 13 13 21
FMM found 1151 interior and 2398 exterior cells
Geometry: AnyGeometry::Convert: FMM grid bounding box -0.0648063 -0.0649216 -0.019575   0.0651937 0.0650784 0.190425
FMM_Fill identifies 1076 surface, 539 interior, 1737 exterior cells
FMM starting with 684 surface cells, grid of size 13 13 21
FMM found 867 interior and 2682 exterior cells
Geometry: AnyGeometry::Convert: FMM grid bounding box -0.065 -0.0662972 -0.193375   0.065 0.0637028 0.016625
FMM_Fill identifies 1503 surface, 788 interior, 2376 exterior cells
FMM starting with 829 surface cells, grid of size 13 31 13
FMM found 1113 interior and 4126 exterior cells
Geometry: AnyGeometry::Convert: FMM grid bounding box -0

In [9]:
# show the world and VSF
from vsf.visualize.klampt_visualization import vsf_to_point_cloud
vis.debug(world=world.world, empty_pc = vsf_to_point_cloud(vsf_empty), origin = view.origin)

Stiffness range:  0.0 0.0
#########################################
klampt.vis: Creating dialog on window 3
#########################################
vis.dialog(): waiting for window 3 on vis thread to complete....
TriMeshTopology: mesh has 9 triangles with duplicate neighbors!
  Triangle range 273 to 924
  May see strange results for some triangle mesh operations
TriMeshTopology: mesh has 104 triangles with duplicate neighbors!
  Triangle range 3828 to 4274
  May see strange results for some triangle mesh operations
TriMeshTopology: mesh has 271 triangles with duplicate neighbors!
  Triangle range 80 to 3362
  May see strange results for some triangle mesh operations
TriMeshTopology: mesh has 635 triangles with duplicate neighbors!
  Triangle range 29 to 3626
  May see strange results for some triangle mesh operations
TriMeshTopology: mesh has 587 triangles with duplicate neighbors!
  Triangle range 1988 to 3472
  May see strange results for some triangle mesh operations
TriMeshTopolo

Now we will set up a dataset. We will load a dataset from a standard folder format, which is configured and loaded automatically in the following example.  We will use a `DatasetConfig` which specifies which keys are present, and which keys used for robot commands and sensor observations.  The `dataset_from_config` function will set up a dataset appropriately to follow the specified configuration.  The standard dataset lazy-loads sequences into memory.

More generally, a dataset is any object that can be treated like a list of sequences, and each sequence is a list of dictionaries mapping keys to numpy arrays.  In other words, datasets behave like a `List[List[Dict[str,np.ndarray]]]` type. If you don't want to go through the trouble of saving such a thing to disk, you can just create such an object yourself.  

In [10]:
from vsf.dataset.constructors import DatasetConfig, dataset_from_config

keys = {'kinova_joint_torques':7,'kinova_state':7}  #describes the keys present in the dataset
dataset_config = DatasetConfig(type='joint_torque_dataset', 
                               path='../demo_data/datasets/rubber_fig_tall_angle00/split1',
                               keys=keys,
                               sensor_keys={'kinova_joint_torques':'kinova_joint_torques'},
                               control_keys={'kinova':'kinova_state'})
dataset = dataset_from_config(dataset_config)
print("Dataset has {} sequences".format(len(dataset)))

Dataset has 32 sequences


For debugging, let's just validate the trajectories coming from the dataset.  Here we'll show the first 5 trajectories.

In [11]:
from klampt.model.trajectory import RobotTrajectory 

for seqno in range(len(dataset)):
    #extract the sequence of commands
    seq = dataset[seqno]
    commands = []
    for frame in range(len(seq)):
        frame = seq[frame]
        commands.append(frame['kinova_state'])

    #convert to a RobotTrajectory and show it
    configs = [robot.configFromDrivers(d) for d in commands]
    traj = RobotTrajectory(robot,[i/len(configs) for i in range(len(configs))],configs)
    vis.debug(robot, {'animation':traj}, 'empty_pc', vsf_to_point_cloud(vsf_empty))
    if seqno >= 4: break

Loading sequence ../demo_data/datasets/rubber_fig_tall_angle00/split1/seq_1704083702
Stiffness range:  0.0 0.0
#########################################
klampt.vis: Creating dialog on window 4
#########################################
vis.dialog(): waiting for window 4 on vis thread to complete....
vis.dialog(): ... dialog done
Loading sequence ../demo_data/datasets/rubber_fig_tall_angle00/split1/seq_1704083759
Stiffness range:  0.0 0.0
#########################################
klampt.vis: Creating dialog on window 5
#########################################
vis.dialog(): waiting for window 5 on vis thread to complete....
vis.dialog(): ... dialog done
Loading sequence ../demo_data/datasets/rubber_fig_tall_angle00/split1/seq_1704083799
Stiffness range:  0.0 0.0
#########################################
klampt.vis: Creating dialog on window 6
#########################################
vis.dialog(): waiting for window 6 on vis thread to complete....
vis.dialog(): ... dialog done
Loading se

We can also check the discrepancy between the observations and the predicted measurements for a random VSF stiffness.  Here, we're also introducing the notion of sensor calibration, which is usually necessary at the beginning of a trial to tare the values coming from the sensor.

A `BaseSensor` can potentially handle its own calibration, but we find it's easier to separate how to calibrate the sensor from the sensor simulator itself.  Here we will generate a `BaseCalibrator` object that we will run at the beginning of each trial to calibrate the sensor.

(There is a little bit of work here to convert the dataset keys to the control and sensor keys expected by the simulator, calibrator, and estimators. This boilerplate is used a lot...)

In [12]:
from vsf.sensor import TareCalibrator
import numpy as np

vsf_empty.stiffness.fill_(0.1)  #set a guessed stiffness of the VSF model
calibrator = TareCalibrator()
for seqno in range(len(dataset)):
    #extract the sequence of controls and observations.  This is boilerplate
    seq = dataset[seqno]
    control_seq = []
    sensor_seq = []
    for frame in seq:
        control_seq.append({k:frame[v] for k,v in dataset_config.control_keys.items()})
        sensor_seq.append({k:frame[v] for k,v in dataset_config.sensor_keys.items()})
        
    #run the calibration
    sim.reset()
    n = calibrator.calibrate(sensor,sim,control_seq,sensor_seq)
    #returns the # of samples used in calibration.  Technically should skip this number of frames for estimation
    print("Sequence",seqno,"calibration:",sensor.get_calibration())

    #now, run the simulator and compare the predicted torques to the actual torques
    dt = 0.1  # a guessed time step.  There's no time-dependent functionality in the quasistatic simulator, so this doesn't matter
    diffs = []
    for frameno in range(n,len(seq)):
        sim.step(control_seq[frameno],dt)
        pred = sim.measurements()['kinova_joint_torques'].numpy()
        actual = sensor_seq[frameno]['kinova_joint_torques']
        assert len(pred) == len(actual)
        diffs.append(pred-actual)
    diffs = np.array(diffs)
    print("Sequence",seqno,"joint torque RMSEs",np.sqrt(np.mean(diffs**2,axis=0)))

    if seqno >= 4: break


Sequence 0 calibration: {'tare': array([ 0.45979138, 18.23648429,  0.42123438,  2.86159472,  0.06669243,
       -0.8061054 , -0.30282573]), 'gravity_pred_at_tare': array([-1.35915481e-04,  1.91127152e+01,  2.38662322e-01,  2.89546379e+00,
        5.89215163e-02, -9.54574667e-01, -7.94720591e-03])}
Sequence 0 joint torque RMSEs [9.17827463 3.28268953 4.65815837 1.8208583  0.497117   0.48569154
 0.09726507]
Sequence 1 calibration: {'tare': array([ 0.52451001, 19.72127437, -0.08059071,  3.76032879,  0.34009876,
       -0.71614274, -0.38496185]), 'gravity_pred_at_tare': array([ 4.59781161e-01,  1.96244829e+01, -3.01500764e-03,  4.47961341e+00,
        3.32438221e-01, -7.43565164e-01, -2.97378133e-01])}
Sequence 1 joint torque RMSEs [5.65413786 2.84083297 3.03784157 0.67209229 0.88973262 0.58248435
 0.26551973]
Sequence 2 calibration: {'tare': array([ 0.37255337, 18.40848739,  0.55852668,  3.35344802,  0.42691417,
       -0.49719183, -0.20882375]), 'gravity_pred_at_tare': array([ 0.06459304

/home/motion/openvsf/demos/../vsf/sensor/base_calibrator.py:78: UserWarning: Contact happens at the first sample, please check the command sequence
  warnings.warn('Contact happens at the first sample, please check the command sequence')


Sequence 3 joint torque RMSEs [3.66120027 1.39108469 1.81534631 0.98915593 0.36882628 0.74600637
 0.06405179]
Sequence 4 calibration: {'tare': array([ 0.44969128, 14.27508675, -0.23326894,  5.27838991,  0.40612126,
       -0.73751265, -0.04340425]), 'gravity_pred_at_tare': array([-0.01852482, 15.00579225, -0.89109171,  5.63442993, -0.02772186,
       -1.27192794, -0.18563287])}
Sequence 4 joint torque RMSEs [0.23552134 0.66598655 0.23684952 0.77890765 0.10500587 0.46168744
 0.11577801]


Now, let's start the estimation process.  Let's start by using *batch* estimation, which uses all the items in the dataset to estimate the VSF stiffness parameters.  We have to provide a prior to the estimator to initialize its stiffness guess, and we will use a GaussianVSFPriorFactory which assigns an independent Gaussian distribution to each point's stiffness.

We can also customize the configuration of the estimator, such as the optimization technique used, but let's not worry about this too much for now.

In [13]:
from vsf.sensor import TareCalibrator
from vsf.estimator.point_vsf_estimator import PointVSFEstimator, PointVSFEstimatorConfig
from vsf.prior.prior_factory import GaussianVSFPriorFactory
import time
import copy
vsf_est = vsf_empty
vsf_empty = copy.deepcopy(vsf_est)  #save a copy of the uninitialized VSF for later

#the second argument sets the prior estimate to a mean of 0.1 and a standard deviation of 1.0
estimator = PointVSFEstimator(PointVSFEstimatorConfig(), GaussianVSFPriorFactory(0.1,1.0))

t0 = time.time()
estimator.batch_estimate(sim, vsf_est, [dataset[i] for i in range(5)], dataset_config, {sensor.name:TareCalibrator()})
t1 = time.time()
print("Estimation took time",t1-t0)

Generating simulation cache
Number of observations: 5
Observed indices: 2070


/home/motion/miniconda3/envs/vsf/lib/python3.10/site-packages/cvxpy/problems/problem.py:158: UserWarning: Objective contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn("Objective contains too many subexpressions. "


QuadProgOptimizer: start solving cvxpy problem...
QuadProgOptimizer: cvxpy solve time = 6.429910898208618
Estimation took time 12.938708066940308


Now let's see some statistics about the estimates.  The VSF stiffness values are in the `stiffness` attribute, and it will also have new two features, 'K_std' and 'N_obs', which give the standard deviations of the estimates and the number of frames in which each point has been touched, respectively.

In [14]:
print('Features:',list(vsf_est.features.keys()))
stiffness = vsf_est.stiffness.cpu().numpy()
N_obs = vsf_est.features['N_obs'].cpu().numpy()
touched_mask = N_obs > 0
print('Stiffness min {}, max {}, mean {}, std {}'.format(np.min(stiffness),np.max(stiffness),np.average(stiffness),np.std(stiffness)))
print('Number of points touched {}/{}'.format(np.sum(N_obs > 0),stiffness.shape[0]))
print('Touched stiffness min {}, max {}, mean {}, std {}'.format(np.min(stiffness[touched_mask]),np.max(stiffness[touched_mask]),np.average(stiffness[touched_mask]),np.std(stiffness[touched_mask])))


Features: ['color', 'K_std', 'N_obs']
Stiffness min -3.1998344623271615e-20, max 10.731247901916504, mean 0.14467070996761322, std 0.411377489566803
Number of points touched 2070/26577
Touched stiffness min -3.1998344623271615e-20, max 10.731247901916504, mean 0.6735327839851379, std 1.3672837018966675


Now let's visualize the estimates. For clearer visualization, we will set the stiffness of the untouched points to 0.

In [15]:
vsf_est.stiffness[~touched_mask] = 0
vis.debug(world=world.world, estimated_pc = vsf_to_point_cloud(vsf_est), origin = view.origin)

Stiffness range:  -3.1998345e-20 10.731248
#########################################
klampt.vis: Creating dialog on window 9
#########################################
vis.dialog(): waiting for window 9 on vis thread to complete....
vis.dialog(): ... dialog done


Finally, let's check the accuracy of the predictions on the 5-sequence training set.

In [16]:
for seqno in range(len(dataset)):
    #extract the sequence of controls and observations.  This is boilerplate
    seq = dataset[seqno]
    control_seq = []
    sensor_seq = []
    for frame in seq:
        control_seq.append({k:frame[v] for k,v in dataset_config.control_keys.items()})
        sensor_seq.append({k:frame[v] for k,v in dataset_config.sensor_keys.items()})
        
    #run the calibration
    sim.reset()
    n = calibrator.calibrate(sensor,sim,control_seq,sensor_seq)

    #now, run the simulator and compare the predicted torques to the actual torques
    dt = 0.1  # a guessed time step.  There's no time-dependent functionality in the quasistatic simulator, so this doesn't matter
    diffs = []
    for frameno in range(n,len(seq)):
        sim.step(control_seq[frameno],dt)
        pred = sim.measurements()['kinova_joint_torques'].numpy()
        actual = sensor_seq[frameno]['kinova_joint_torques']
        assert len(pred) == len(actual)
        diffs.append(pred-actual)
    diffs = np.array(diffs)
    print("Sequence",seqno,"joint torque RMSEs",np.sqrt(np.mean(diffs**2,axis=0)))

    if seqno >= 4: break

Sequence 0 joint torque RMSEs [0.65597066 3.56731717 1.10205706 1.93419589 0.84548271 0.3813909
 0.148807  ]
Sequence 1 joint torque RMSEs [2.09448144 1.14818954 1.24078013 1.01514761 0.67603356 0.36889048
 0.26586546]
Sequence 2 joint torque RMSEs [3.33935766 0.61217771 1.31126004 0.93945916 0.27872824 0.18159706
 0.28477131]
Sequence 3 joint torque RMSEs [1.9271775  0.64566873 1.4919207  0.84887023 0.38229206 0.75245997
 0.0639786 ]
Sequence 4 joint torque RMSEs [0.30714597 0.62065842 0.25630812 0.78157091 0.10857327 0.48224374
 0.10941095]


Since comparisons are done frequently, we also have standard functions for calculating these metrics.

In [17]:
from vsf.sim.metrics import rmse_sensors
calibrators = {'kinova_joint_torques':calibrator}
rmses = rmse_sensors(sim,[dataset[i] for i in range(5)],dataset_config,calibrators)
for i in range(5):
    print("Sequence",i,"joint torque RMSEs",rmses['kinova_joint_torques'][i])
sim.vsf_objects['vsf'].vsf_model = vsf_empty
print("Original RMSE train",rmse_sensors(sim,[dataset[i] for i in range(5)],dataset_config,calibrators,aggregate_seqs=True,aggregate_channels=True)['kinova_joint_torques'])
print("Original RMSE test",rmse_sensors(sim,[dataset[i] for i in range(5,30)],dataset_config,calibrators,aggregate_seqs=True,aggregate_channels=True)['kinova_joint_torques'])
sim.vsf_objects['vsf'].vsf_model = vsf_est
print("Estimated RMSE train",rmse_sensors(sim,[dataset[i] for i in range(5)],dataset_config,calibrators,aggregate_seqs=True,aggregate_channels=True)['kinova_joint_torques'])
print("Estimated RMSE test",rmse_sensors(sim,[dataset[i] for i in range(5,30)],dataset_config,calibrators,aggregate_seqs=True,aggregate_channels=True)['kinova_joint_torques'])

Sequence 0 joint torque RMSEs [0.69780016 3.84516329 1.61585165 2.01967389 1.23613667 0.8431708
 0.25137202]
Sequence 1 joint torque RMSEs [2.20845026 1.00239434 1.20029741 1.14892304 0.6970521  0.60164085
 0.39099949]
Sequence 2 joint torque RMSEs [3.47797904 0.68699538 1.53716045 1.22286451 0.61877105 0.44399902
 0.16159432]
Sequence 3 joint torque RMSEs [1.89585819 0.66626561 1.15412607 0.78892602 0.14895561 0.23488888
 0.16254136]
Sequence 4 joint torque RMSEs [0.33031763 1.28129661 0.59709474 0.92900601 0.42794409 0.0803795
 0.17714087]
Original RMSE train 2.4999126840715498
Loading sequence ../demo_data/datasets/rubber_fig_tall_angle00/split1/seq_1704083931
Loading sequence ../demo_data/datasets/rubber_fig_tall_angle00/split1/seq_1704083986
Loading sequence ../demo_data/datasets/rubber_fig_tall_angle00/split1/seq_1704084035
Loading sequence ../demo_data/datasets/rubber_fig_tall_angle00/split1/seq_1704084077
Loading sequence ../demo_data/datasets/rubber_fig_tall_angle00/split1/seq

Now we see that we've done a much better job of estimating the RMSEs on the training set but we haven't really improved the testing RMSEs very much.  This is because we've only used 5 sequences for training, and they may have not done a good job of covering the object.  This is a typical issue with using touch data alone.  Read on to the next section to explore better ways of extrapolating using vision features and priors. 

## Extrapolating VSF data by color

This section provides a basic demonstration of learning a color-conditioned stiffness prior.
We use a simple algorithm that naively regresses the estimated VSF stiffness using stochastic gradient descent.
For more advanced and accurate prior learning, please refer to the Bayesian meta-learning function in estimator module.

In [18]:
import torch
from vsf.prior.conditional_distribution import LinearGaussianPriorConfig, LinearGaussianPrior
from vsf.prior.prior_factory import LearnableVSFPriorFactory

linear_gaussian_condig = LinearGaussianPriorConfig(c_dim=3, diag=True, non_neg=True)

# setup a simple meta-prior using linear Gaussian with RGB as input features
prior_factory = LearnableVSFPriorFactory(['color'], LinearGaussianPrior(linear_gaussian_condig))

# TODO: point vsf does not have consistent dtype, need manually convert to float
vsf_est.features['color'] = vsf_est.features['color'].float()

prior_factory.meta_learn(vsfs=[vsf_est], features=[{'color':vsf_est.features['color'].float()}])

In [19]:
# Now create an estimator using the learned prior
estimator = PointVSFEstimator(PointVSFEstimatorConfig(), prior_factory)

t0 = time.time()
estimator.batch_estimate(sim, vsf_est, [dataset[i] for i in range(5)], dataset_config, {sensor.name:TareCalibrator()})
t1 = time.time()
print("Estimation took time",t1-t0)

Generating simulation cache


/home/motion/openvsf/demos/../vsf/sensor/base_calibrator.py:78: UserWarning: Contact happens at the first sample, please check the command sequence
  warnings.warn('Contact happens at the first sample, please check the command sequence')


Number of observations: 5
Observed indices: 2070


/home/motion/miniconda3/envs/vsf/lib/python3.10/site-packages/cvxpy/problems/problem.py:158: UserWarning: Objective contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn("Objective contains too many subexpressions. "


QuadProgOptimizer: start solving cvxpy problem...
QuadProgOptimizer: cvxpy solve time = 6.372997045516968
Estimation took time 12.989048957824707


In [20]:
# show results with color prior, we expect to see a larger high stiffness region 
vsf_show(vsf_est)

Setting voxel grid resolution 0.003220559224726908
vsf_show: Using the following stiffness values: [0.010435911826789379, 0.03983685374259949, 0.06923779100179672, 0.09863872826099396, 0.1280396729707718]
klampt.vis: auto-fitting camera to scene.


Unrecognized OpenGL version
Unrecognized OpenGL version


## Estimating VSFs for Moving Objects

PointVSF also supports estimation when the object is moving.

We assume the presence of an external tracking module that estimates the current rigid transformation of the VSF object. The only difference in this setting is that, when applying control to the object, an additional key `{vsf_object_name}_state` must be provided. This key represents the homogeneous transformation of the object as a 4x4 matrix. The simulator will internally update the rigid transformation of the VSF object before running the simulation.

To estimate a movable VSF, load the `brown_boot_moving` object.

In [21]:
# load empty vsf model for moving brown boot
from vsf import vsf_from_file
moving_vsf_empty = vsf_from_file('../demo_data/saved_vsfs/brown_boot_moving/vsf_empty.npz')

In [22]:
# Show the processed VSF model
from vsf.visualize.klampt_visualization import vsf_to_point_cloud
vis.debug(empty_pc = vsf_to_point_cloud(moving_vsf_empty, masked_view_fraction=1.0))

Stiffness range:  0.0 0.0
#########################################
klampt.vis: Creating dialog on window 10
#########################################
vis.dialog(): waiting for window 10 on vis thread to complete....
vis.dialog(): ... dialog done


Set up the simulator for a movable VSF object. This setup is identical to creating a standard simulation environment, except for the control keys used to manipulate the object, which will be introduced in the next section on dataset setup.

In [23]:
from vsf.sim import klamptWorldWrapper, QuasistaticVSFSimulator
from vsf.sim.point_vsf_body import ContactParams
from vsf.sensor.punyo_dense_force_sensor import PunyoDenseForceSensor

# create a new world
world = klamptWorldWrapper()
world.add_robot('kinova','../knowledge/robot_model/kinova_gen3.urdf')
robot = world.world.robot(0)

# the punyo geometry is added to the end effector link of the robot
world.add_geometry_from_file('punyo', '../knowledge/robot_model/punyo_mesh_complete.ply', 
                             geom_type='deformable', parent_name='EndEffector_Link', 
                             parent_relative_transform=np.array([[0.0, -1.0, 0.0, 0.0], [0.34202014, 0.0, -0.93969262, 0.02111], 
                                                                 [0.93969262, 0.0, 0.34202014, 0.096142], [0.0, 0.0, 0.0, 1.0]]))

# this preprocessing needs to be done before running the simulator with a point VSF body
world.setup_local_pcd_lst('open3d')
world.setup_local_sdf_lst()

# initialize all sensors to add in the simulator
dense_force_sensor = PunyoDenseForceSensor('punyo_force','punyo')

# create a simulator with the world, a joint torque sensor, and the vsf body
moving_sim = QuasistaticVSFSimulator(world, [dense_force_sensor])
vsf_body = moving_sim.add_deformable('vsf', moving_vsf_empty, contact_params=None)  #if you want to customize how the VSF is simulated, you can pass in a ContactParams object here

WorldModel::LoadRobot: ../knowledge/robot_model/kinova_gen3.urdf
URDFParser: Link size: 9
URDFParser: Joint size: 9
TriMeshTopology: mesh has 7 triangles with duplicate neighbors!
  Triangle range 2410 to 2722
  May see strange results for some triangle mesh operations
TriMeshTopology: mesh has 7 triangles with duplicate neighbors!
  Triangle range 2410 to 2722
  May see strange results for some triangle mesh operations
TriMeshTopology: mesh has 22 triangles with duplicate neighbors!
  Triangle range 1990 to 3298
  May see strange results for some triangle mesh operations
TriMeshTopology: mesh has 22 triangles with duplicate neighbors!
  Triangle range 1990 to 3298
  May see strange results for some triangle mesh operations
TriMeshTopology: mesh has 18 triangles with duplicate neighbors!
  Triangle range 4061 to 4257
  May see strange results for some triangle mesh operations
TriMeshTopology: mesh has 18 triangles with duplicate neighbors!
  Triangle range 4061 to 4257
  May see strang

Set Up the Dataset with a Moving Object

Since we use the `PunyoDenseForceSensor`, we obtain the deformed vertex locations as `punyo_state` and the per-vertex contact forces as `punyo_force`.
The `vsf_state` represents the homogeneous rigid transformation of the VSF object, which is obtained from an external tracking algorithm.

In [24]:
from vsf.dataset.constructors import DatasetConfig, dataset_from_config

keys = {'kinova_state': 7, 
        'punyo_state': [[390, 3], np.ndarray], 
        'punyo_force': [[390, 3], np.ndarray], 
        'vsf_state': [[4, 4], np.ndarray]}  # vsf_state is rigid transformation of VSF object
dataset_config = DatasetConfig(type='moving_dataset', 
                               path='../demo_data/datasets/brown_boot_moving', keys=keys)
dataset = dataset_from_config(dataset_config)
print("Dataset has {} sequences".format(len(dataset)))

Dataset has 90 sequences


Here we visualize the collected  dataset with estimated tracked object

In [ ]:
from klampt.math import se3
from klampt import vis
vis.init('PyQt')  #needed inside Jupyter Notebook to show an OpenGL window
vis.scene().clear()
vis.add("world", world.world)


# add table
# the table are for visualization only, not added to the simulator to avoid extra computation in the simulation
from klampt.model.create import box
b1 = box(1.,1.5,1.1,center=(0.25,0,-0.55),type='GeometricPrimitive')
vis.add("table", b1, hide_label=True)
b2 = box(0.7,0.45,0.12,center=(0.3,0.4,0),type='GeometricPrimitive')
vis.add("box", b2, hide_label=True)

# add triangle mesh of the moving brown boot to visualize tracked transformations
boot_mesh = Geometry3D()
boot_mesh.loadFile('../demo_data/datasets/brown_boot_moving/object/mesh.obj')

vis.add("boot_mesh", boot_mesh)
vis.show()

import time
for i in range(len(dataset)):
    seq = dataset[i]
    for frame in seq:
        control = {}
        control['kinova'] = frame['kinova_state']
        control['punyo'] = frame['punyo_state']

        # step simulation
        vis.lock()
        moving_sim.step(control, 0.1)
        boot_mesh.setCurrentTransform(*se3.from_ndarray(frame['vsf_state'])) # object mesh is for visualization only, so not added to the simulator
        vis.unlock()

        time.sleep(2/len(seq)) # visualize each sequence for 2 second

    moving_sim.reset()
    if i >= 5:
        break
vis.clear()
vis.kill()

Run Point VSF estimation on a moving object. Since we use the Punyo Dense Force Sensor, the estimation may take longer.

In this example, we run the estimation on a single touch sequence to demonstrate that the estimation code remains the same as in the previous pipeline with a fixed VSF object.

In [25]:
from vsf.sensor import TareCalibrator
from vsf.estimator.point_vsf_estimator import PointVSFEstimator, PointVSFEstimatorConfig
from vsf.prior.prior_factory import GaussianVSFPriorFactory
import time
import copy
moving_vsf_est = moving_vsf_empty
moving_vsf_empty = copy.deepcopy(moving_vsf_est)  #save a copy of the uninitialized VSF for later

#the second argument sets the prior estimate to a mean of 0.5 and a standard deviation of 1.0
estimator = PointVSFEstimator(PointVSFEstimatorConfig(), GaussianVSFPriorFactory(0.0,1.0))

t0 = time.time()
estimator.batch_estimate(moving_sim, moving_vsf_est, [dataset[i] for i in range(1)], dataset_config)
t1 = time.time()
print("Estimation took time",t1-t0)

Loading sequence ../demo_data/datasets/brown_boot_moving/seq_000
Generating simulation cache
Number of observations: 1
Observed indices: 43837
QuadProgOptimizer: start solving cvxpy problem...
QuadProgOptimizer: cvxpy solve time = 30.95504140853882
Estimation took time 51.40388250350952


In [26]:
# visualize estimated vsf
# point VSF estimation with Punyo dense force sensor suffer from resolution limitation
# please refer to the neural_vsf_playground notebook for better estimation with neural VSF
vsf_show(moving_vsf_est)

Setting voxel grid resolution 0.0019416289327088357
vsf_show: Using the following stiffness values: [2.5153495695619623e-16, 0.0007477474282495677, 0.0014954948564991355, 0.0022432422265410423, 0.002990989712998271]
klampt.vis: auto-fitting camera to scene.


Unrecognized OpenGL version
Unrecognized OpenGL version


## Multi-sensor estimation

The estimation techniques above can work for any number of sensors.  Simply create more sensors to add to the simulator, and set up the observations dictionary to the estimator appropriately.    

In [27]:
from vsf import vsf_from_file
shoe_vsf_empty = vsf_from_file('../demo_data/saved_vsfs/white_nike_fixed/vsf_empty.npz')

In [28]:
# Show the processed VSF model
from vsf.visualize.klampt_visualization import vsf_to_point_cloud
vis.debug(empty_pc = vsf_to_point_cloud(shoe_vsf_empty, masked_view_fraction=1.0))

Stiffness range:  0.0 0.0
#########################################
klampt.vis: Creating dialog on window 11
#########################################
vis.dialog(): waiting for window 11 on vis thread to complete....
vis.dialog(): ... dialog done


In this example, we add a joint torque sensor and a Punyo pressure sensor to the simulator.

In [29]:
from vsf.sim import klamptWorldWrapper, QuasistaticVSFSimulator
from vsf.sim.point_vsf_body import ContactParams
from vsf.sensor.joint_torque_sensor import JointTorqueSensor
from vsf.sensor.punyo_pressure_sensor import PunyoPressureSensor

# create a new world
world = klamptWorldWrapper()
world.add_robot('kinova','../knowledge/robot_model/kinova_gen3.urdf')
robot = world.world.robot(0)

# the punyo geometry is added to the end effector link of the robot
world.add_geometry_from_file('punyo', '../knowledge/robot_model/punyo_mesh_complete.ply', 
                             geom_type='deformable', parent_name='EndEffector_Link', 
                             parent_relative_transform=np.array([[0.0, -1.0, 0.0, 0.0], 
                                                                 [-0.05582150, 0.0, -0.9984407, 0.02111], 
                                                                 [0.99844076, 0.0, -0.0558215, 0.096142], 
                                                                 [0.0, 0.0, 0.0, 1.0]]))

# this preprocessing needs to be done before running the simulator with a point VSF body
world.setup_local_pcd_lst('open3d')
world.setup_local_sdf_lst()

# initialize all sensors to add in the simulator
joint_torque_sensor = JointTorqueSensor('kinova_joint_torques','kinova',[robot.link(i).name for i in range(1,8)])
pressure_sensor = PunyoPressureSensor('punyo_pressure','punyo')

# create a simulator with the world, a joint torque sensor, and the vsf body
shoe_sim = QuasistaticVSFSimulator(world, [joint_torque_sensor, pressure_sensor])
vsf_body = shoe_sim.add_deformable('vsf', shoe_vsf_empty, contact_params=None)  #if you want to customize how the VSF is simulated, you can pass in a ContactParams object here

WorldModel::LoadRobot: ../knowledge/robot_model/kinova_gen3.urdf
URDFParser: Link size: 9
URDFParser: Joint size: 9
URDFParser: Done loading robot file ../knowledge/robot_model/kinova_gen3.urdf
FMM_Fill identifies 1305 surface, 731 interior, 2003 exterior cells
FMM starting with 835 surface cells, grid of size 13 13 21
FMM found 1151 interior and 2398 exterior cells
FMM_Fill identifies 1076 surface, 539 interior, 1737 exterior cells
FMM starting with 684 surface cells, grid of size 13 13 21
FMM found 867 interior and 2682 exterior cells
FMM_Fill identifies 1503 surface, 788 interior, 2376 exterior cells
FMM starting with 829 surface cells, grid of size 13 31 13
FMM found 1113 interior and 4126 exterior cells
FMM_Fill identifies 1358 surface, 699 interior, 2167 exterior cells
FMM starting with 781 surface cells, grid of size 13 13 28
FMM found 1077 interior and 3655 exterior cells
FMM_Fill identifies 1122 surface, 526 interior, 1868 exterior cells
FMM starting with 686 surface cells, gr

Setup the dataset for multiple sensors.

In [30]:
from vsf.dataset.constructors import DatasetConfig, dataset_from_config

keys = {'kinova_joint_torques': 7, 'kinova_state': 7, 'punyo_pressure': 1}  #describes the keys present in the dataset
dataset_config = DatasetConfig(type='multi_sensor_dataset', 
                               path='../demo_data/datasets/white_nike_fixed', keys=keys)
dataset = dataset_from_config(dataset_config)
print("Dataset has {} sequences".format(len(dataset)))

Dataset has 181 sequences


Here the estimation code is basically the same, the only difference is that we need to provide each sensor a corresponding calibrator.

In [31]:
from vsf.sensor import TareCalibrator
from vsf.estimator.point_vsf_estimator import PointVSFEstimator, PointVSFEstimatorConfig
from vsf.prior.prior_factory import GaussianVSFPriorFactory
import time
import copy
shoe_vsf_est = shoe_vsf_empty
shoe_vsf_empty = copy.deepcopy(shoe_vsf_est)  #save a copy of the uninitialized VSF for later

#the second argument sets the prior estimate to a mean of 0.5 and a standard deviation of 1.0
estimator = PointVSFEstimator(PointVSFEstimatorConfig(), GaussianVSFPriorFactory(0.0,1.0))

calibrators = {'kinova_joint_torques':TareCalibrator(), 'punyo_pressure':TareCalibrator()}

t0 = time.time()
estimator.batch_estimate(shoe_sim, shoe_vsf_est, [dataset[i] for i in range(5)], dataset_config, calibrators)
t1 = time.time()
print("Estimation took time",t1-t0)

Loading sequence ../demo_data/datasets/white_nike_fixed/seq_000
Loading sequence ../demo_data/datasets/white_nike_fixed/seq_001
Loading sequence ../demo_data/datasets/white_nike_fixed/seq_002
Loading sequence ../demo_data/datasets/white_nike_fixed/seq_003
Loading sequence ../demo_data/datasets/white_nike_fixed/seq_004
Generating simulation cache


/home/motion/openvsf/demos/../vsf/sensor/base_calibrator.py:78: UserWarning: Contact happens at the first sample, please check the command sequence
  warnings.warn('Contact happens at the first sample, please check the command sequence')


Number of observations: 5
Observed indices: 2311
QuadProgOptimizer: start solving cvxpy problem...
QuadProgOptimizer: cvxpy solve time = 0.6255617141723633
Estimation took time 1.2020339965820312


In [32]:
# visualize estimated vsf
vsf_show(shoe_vsf_est)

Setting voxel grid resolution 0.008660254037844345
vsf_show: Using the following stiffness values: [8.399690833170389e-20, 0.03892963379621506, 0.07785926759243011, 0.11678890138864517, 0.15571853518486023]
klampt.vis: auto-fitting camera to scene.


Unrecognized OpenGL version
Unrecognized OpenGL version


In [33]:
from vsf.utils.data_utils import remap_dict_in_seq

for seqno in range(len(dataset)):
    #extract the sequence of controls and observations.  This is boilerplate
    seq = dataset[seqno]
    
    control_seq, sensor_seq = remap_dict_in_seq(seq, shoe_sim.get_control_keys(), shoe_sim.get_sensor_keys())
        
    #run the calibration
    shoe_sim.reset()
    n = max(calibrators['kinova_joint_torques'].calibrate(joint_torque_sensor,shoe_sim,control_seq,sensor_seq), 
            calibrators['punyo_pressure'].calibrate(pressure_sensor,shoe_sim,control_seq,sensor_seq))

    #now, run the simulator and compare the predicted torques to the actual torques
    dt = 0.1  # a guessed time step.  There's no time-dependent functionality in the quasistatic simulator, so this doesn't matter
    joint_torque_diffs = []
    punyo_pressure_diffs = []
    for frameno in range(n,len(seq)):
        shoe_sim.step(control_seq[frameno],dt)
        joint_torque_pred = shoe_sim.measurements()['kinova_joint_torques'].numpy()
        joint_torque_actual = sensor_seq[frameno]['kinova_joint_torques']
        assert len(joint_torque_pred) == len(joint_torque_actual)
        joint_torque_diffs.append(joint_torque_pred-joint_torque_actual)
        
        punyo_pressure_pred = shoe_sim.measurements()['punyo_pressure'].numpy()
        punyo_pressure_actual = sensor_seq[frameno]['punyo_pressure']
        assert len(punyo_pressure_pred) == len(punyo_pressure_actual)
        punyo_pressure_diffs.append(punyo_pressure_pred-punyo_pressure_actual)

    joint_torque_diffs = np.array(joint_torque_diffs)
    print("Sequence",seqno,"joint torque RMSEs",np.sqrt(np.mean(joint_torque_diffs**2,axis=0)))
    
    punyo_pressure_diffs = np.array(punyo_pressure_diffs)
    print("Sequence",seqno,"punyo pressure RMSEs",np.sqrt(np.mean(punyo_pressure_diffs**2,axis=0)))

    if seqno >= 4: break

Sequence 0 joint torque RMSEs [3.08475313 4.34439077 2.97743361 2.10242783 0.25027262 2.40612587
 0.18195641]
Sequence 0 punyo pressure RMSEs [0.07472547]
Sequence 1 joint torque RMSEs [2.78579017 3.79814828 1.98058953 4.54617965 0.31258196 2.55038131
 0.32657833]
Sequence 1 punyo pressure RMSEs [0.1162529]
Sequence 2 joint torque RMSEs [2.16866864 5.07206635 1.90121116 3.20353391 0.41221836 2.59197523
 0.20635116]
Sequence 2 punyo pressure RMSEs [0.0791336]
Sequence 3 joint torque RMSEs [1.68484687 4.30863297 0.88164929 6.09108716 0.09132125 2.9835171
 0.35279591]
Sequence 3 punyo pressure RMSEs [0.14270338]
Sequence 4 joint torque RMSEs [3.23590293 5.74930225 2.10608991 7.87075234 0.25791414 4.08796978
 0.10874685]
Sequence 4 punyo pressure RMSEs [0.27086107]
